In [242]:
import zarr
import pysam
import numpy as np

In [243]:
m = np.array([1,2,4])
n = np.array([1,3,4])

np.stack((n,m), axis=0)

array([[1, 3, 4],
       [1, 2, 4]])

In [244]:
store = zarr.storage.MemoryStore()
z = zarr.create_array(store=store, shape=(10, 0), chunks=(10, 100), dtype='int32')
z.shape



(10, 0)

In [245]:
m = np.arange(200).reshape((10,20))
z.append(m, axis = 1)
z.shape

(10, 20)

In [246]:

z2 = zarr.create_array(zarr.storage.MemoryStore(), shape=(1,0), chunks=(1,10), dtype='int32')
z2.append(np.array([1,2]).reshape(1,-1), axis=1)
z2[:,:]

array([[1, 2]], dtype=int32)

In [247]:
z[:,:]

array([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19],
       [ 20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,
         33,  34,  35,  36,  37,  38,  39],
       [ 40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
         53,  54,  55,  56,  57,  58,  59],
       [ 60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,
         73,  74,  75,  76,  77,  78,  79],
       [ 80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
         93,  94,  95,  96,  97,  98,  99],
       [100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
        113, 114, 115, 116, 117, 118, 119],
       [120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132,
        133, 134, 135, 136, 137, 138, 139],
       [140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
        153, 154, 155, 156, 157, 158, 159],
       [160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 1

In [248]:
da1 = zarr.open_group('../data/01_processed/ssl_sets/da1_subset_10k.zarr')

In [249]:
read=50
da1['data'][:,int(da1['indptr'][read]):int(da1['indptr'][read+1])]

array([[ 0,  2,  2, ...,  2,  1,  1],
       [33, 36, 40, ..., 24, 46, 18],
       [11, 15, 18, ...,  7, 30, 33],
       [ 8, 43, 15, ..., 69,  9, 35],
       [33, 33,  8, ..., 20, 14, 15],
       [93, 44, 93, ..., 93, 93, 93]], shape=(6, 21159), dtype=uint8)

In [250]:
da1['indptr'][1]


array(20986, dtype=uint32)

In [251]:
da1['data'].shape

(6, 171219240)

In [252]:
171219240/10_000

17121.924

In [253]:
x = [1,2] + [2,3]
print(x)

[1, 2, 2, 3]


In [294]:
(300*(8_800_000/200_000))/(60*60)

3.6666666666666665

In [255]:
ob007_no_shards = zarr.open_group('../data/01_processed/ssl_sets/ob007_20mchunk_noshard.zarr')
ob007_with_shards = zarr.open_group('../data/01_processed/ssl_sets/ob007.zarr')


In [256]:
x = ob007_no_shards['data'][:,21_000_000:22_000_000]
x.shape

(8, 1000000)

In [257]:
x = ob007_with_shards['data'][:,21_000_000:22_000_000]

In [258]:
x.shape[1]/8_0000

12.5

In [259]:
x[:,10_020_000:10_040_000]

array([], shape=(8, 0), dtype=uint8)

In [260]:
read=54_000
ob007['data'][:,int(ob007['indptr'][read]):int(ob007['indptr'][read+1])]

array([[ 2,  2,  1, ...,  3,  1,  1],
       [20, 16,  6, ..., 11,  7,  6],
       [20, 21, 17, ..., 26, 73, 30],
       ...,
       [86, 93, 93, ..., 93, 93, 93],
       [14, 16, 16, ..., 18, 18, 16],
       [ 1,  0,  0, ...,  0,  0,  1]], shape=(8, 12915), dtype=uint8)

In [261]:
start_read = 50_000
reads = 10_000
x=ob007['data'][:,int(ob007['indptr'][start_read]):int(ob007['indptr'][start_read+reads])]

In [262]:
x[:,100:40_00_000]

array([[ 0,  0,  3, ...,  3,  3,  3],
       [10,  9, 15, ..., 12,  4, 14],
       [27, 18, 14, ..., 10, 11, 35],
       ...,
       [93, 93, 93, ..., 93, 93, 60],
       [60, 59, 55, ...,  9,  9,  8],
       [ 0,  1,  1, ...,  0,  0,  0]], shape=(8, 3999900), dtype=uint8)

In [263]:
# mean read length
np.diff(ob007['indptr']).mean()

np.float64(8123.962086513995)

In [264]:

import time

def benchmark_indptr_read(path, n_batches=20, batch_size=128):
    backend='zarr3'
    
    # 1. Open Dataset
    try:
        group = zarr.open_group(path, mode='r')
        arr = group['data']
        indptr = group['indptr']
    except Exception as e:
        print(f"Error opening zarr: {e}")
        return

    print(arr.shape)
    # Select random STARTING sequences
    # e.g., if we need 32 sequences, we pick 32 random indices from indptr
    batch_indices = np.random.randint(0, indptr.shape[0] - 2, size=(n_batches, batch_size))
    print(batch_indices.shape)
    total_bytes = 0
    start_time = time.time()
    
    for i in range(n_batches):
        # Current batch of sequence indices
        seq_idxs = batch_indices[i]
        
        # In a real dataloader, this part happens in the worker process
        for seq_id in seq_idxs:
            # A. LOOKUP (In Memory - Fast)
            start = indptr[seq_id]
            end = indptr[seq_id+1]
            
            # B. READ (From Disk - Slow)
            if backend == 'zarr':
                _ = arr[start:end]
            elif backend == 'tensorstore':
                # .read().result() blocks until data is loaded
                _ = arr[start:end].read().result()
            
            # Calculate bytes (approx based on uint8 * num_features)
            # We assume num_features is the second dim
            total_bytes += (end - start) * arr.shape[1]

    end_time = time.time()
    total_time = end_time - start_time
    
    mb_read = total_bytes / 1024 / 1024
    throughput = mb_read / total_time
    
    print(f"  > Total Read:  {mb_read:.2f} MB")
    print(f"  > Time Taken:  {total_time:.4f} s")
    print(f"  > Throughput:  {throughput:.2f} MB/s")

benchmark_indptr_read("../data/01_processed/ssl_sets/ob007_no_shards.zarr")


(1596358550, 8)
(20, 128)
  > Total Read:  161.22 MB
  > Time Taken:  11.5737 s
  > Throughput:  13.93 MB/s


In [265]:
n_batches = 20
batch_size=32
path = "../data/01_processed/ssl_sets/ob007_no_shards.zarr"
group = zarr.open_group(path, mode='r')
arr = group['data']
indptr = group['indptr']
print(arr.shape)

batch_indices = np.random.randint(0, indptr.shape[0] - 2, size=(n_batches, batch_size))
print(batch_indices.shape)
print([indptr[batch_indices]])
total_bytes = 0
start_time = time.time()

slices = [arr[indptr[i]:indptr[i+1]] for i in batch_indices]

(1596358550, 8)
(20, 32)
[array([[1202984159,  398081438, 1171842333,  788178001,  414463086,
         135410407, 1008686920, 1010988990, 1215364123, 1227164950,
         121203324,  298271676, 1439648611,  206026887,  928608050,
         573819350, 1285910634,  335592211, 1451380530,  807935321,
        1414930295, 1369669774,    6151992,  491385923,  814962298,
        1194421690, 1133937580,  277032647,  302042842, 1552853440,
        1542584911, 1505262715],
       [ 406029653, 1078488457,  559661529, 1156330518, 1410935788,
         510142580,  925209567,   25590750,  615585806, 1176110408,
         371872630,  155322109,  950831502, 1232607611, 1155510096,
         460845056, 1187089153, 1131398660,  413728954,  367347514,
        1366168868,  245380409,   26443233, 1183127668,  832761378,
        1378538345, 1387954041,  733797327,  927347293, 1175696858,
         878717226,  725546758],
       [1550205909,   13937086,  358192037,   44445391, 1530634748,
         782952424, 1053

TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
import tensorstore as ts
import numpy as np
import asyncio

arr_schema = {
    'driver': 'zarr3',
    'kvstore': {'driver': 'file'},  # Using memory for demo, normally 'file' or 's3'
    'metadata': {'shape': [10**8], 'chunks': [10**6], 'dtype': '<i8'}
}

async def get_slices_parallel(indptr, indices):
    # 1. Open the store asynchronously
    dataset = await ts.open(arr_schema)

    # 2. Calculate starts and stops
    # indptrs are likely in memory, so this is fast
    starts = indptr[indices]
    stops = indptr[indices + 1]

    # 3. Dispatch all reads immediately
    # We create a list of futures. TensorStore executes these in parallel 
    # using its internal thread pool.
    futures = []
    for start, stop in zip(starts, stops):
        # We ask for a numpy view of the result
        futures.append(dataset[start:stop].read())

    # 4. Wait for all I/O to complete
    # 'gather' runs them concurrently
    arrays = await asyncio.gather(*futures)

    # 5. Concatenate final result
    return np.concatenate(arrays)

In [ ]:
arr = np.array([100, 203, 500])

# Stack creates pairs [[100, 101], [203, 204], ...], ravel flattens them
result = np.stack((arr, arr + 1), axis=-1).ravel()
result

array([100, 101, 203, 204, 500, 501])

In [ ]:
import numpy as np
import time

# Create a large array (100 million floats ~ 800MB)
arr = np.arange(10**8, dtype=np.float64)

# Method 1: np.r_ (The "Slow" Way)
t0 = time.time()
# Selects two 10-million element chunks
indices = np.r_[0:10**7, 5*10**7:6*10**7] 
res_r = arr[indices]
t1 = time.time()
print(f"np.r_:        {t1 - t0:.4f} seconds")

# Method 2: np.concatenate (The "Fast" Way)
t0 = time.time()
# Creates views first, no intermediate index array
res_c = np.concatenate((arr[0:10**7], arr[5*10**7:6*10**7]))
t1 = time.time()
print(f"concatenate:  {t1 - t0:.4f} seconds")

del arr, res_r, res_c, indices

np.r_:        0.2898 seconds
concatenate:  0.0492 seconds


In [ ]:

import torch
import tensorstore as ts
import numpy as np

class TensorStoreDataset(torch.utils.data.Dataset):
    def __init__(self, zarr_path):
        # Open the store. This is lightweight (lazy).
        # We specify the context to ensure threads are managed correctly.
        self.store = ts.open({
            'driver': 'zarr3',
            'kvstore': {'driver': 'file', 'path': zarr_path},
        }).result()
        
        self.indptr =self.store['indptr']

    def __getitem__(self, indices):
        # indices is a LIST of integers (provided by BatchSampler)
        
        # 1. Resolve starts/ends for the batch
        # Assuming indptrs defines variable-length rows
        starts = self.indptr[indices]
        ends = self.indptr[indices + 1]
        
        # 2. Fire off async reads immediately
        # TensorStore will coalesce reads and run them in its internal C++ thread pool.
        futures = []
        for start, end in zip(starts, ends):
            # Request the slice. This returns a Future immediately.
            futures.append(self.store['data'][start:end].read())

        # 3. Block and gather results
        # This releases the GIL for the heavy I/O and decompression work in C++
        arrays = [f.result() for f in futures]
        
        # 4. Concatenate and return
        # This happens in the worker process, so the main process just gets the final tensor
        batch_data = np.concatenate(arrays)
        return batch_data

    def __len__(self):
        return len(self.indptr) - 1

In [ ]:
np.max(np.diff(ob007['indptr']))

np.uint64(30895)

In [ ]:
np.min(np.diff(da1['indptr']))

np.uint32(3592)

In [2]:
import tensorstore as ts
import torch
import os
from torch.utils.data import Dataset, DataLoader

class ZarrRaggedDataset(Dataset):
    def __init__(self, group_path, data_key='data', indptr_key='indptr', context_spec=None):
        ctx = ts.Context(context_spec) if context_spec else None
        abs_path = os.path.abspath(group_path)

        # 1. Setup indptr spec
        spec_indptr = {
            'driver': 'zarr', 
            'kvstore': {
                'driver': 'file', 
                'path': os.path.join(abs_path, indptr_key)
            }
        }
        
        # CORRECTED CHAIN:
        # ts.open(...) returns a Future. 
        # .result() blocks to get the TensorStore object.
        # .read() returns a Future.
        # .result() blocks to get the numpy array.
        self.indptr = ts.open(spec_indptr, context=ctx).result().read().result()
        
        # 2. Setup data spec
        spec_data = {
            'driver': 'zarr', 
            'kvstore': {
                'driver': 'file', 
                'path': os.path.join(abs_path, data_key)
            }
        }
        
        # We only resolve the store here, we don't read the data yet
        self.data_store = ts.open(spec_data, context=ctx).result()

    def __len__(self):
        return len(self.indptr) - 1

    def __getitem__(self, idx):
        start = self.indptr[idx]
        end = self.indptr[idx + 1]
        
        # Read the specific slice
        val = self.data_store[start:end].read().result()
        return torch.from_numpy(val)

In [3]:
ds = ZarrRaggedDataset(path)

NameError: name 'path' is not defined

In [4]:
ds[100,101].shape

NameError: name 'ds' is not defined

In [ ]:
def get_dataloader(path, batch_size=32, num_workers=4):
    # Optimize TensorStore context for the H100 environment
    # 'cache_pool': controls memory usage for cached chunks
    ts_context = {
        'cache_pool': {'total_bytes_limit': 4_000_000_000}, # 4GB cache
        'data_copy_concurrency': {'limit': num_workers * 2},
    }
    
    dataset = ZarrRaggedDataset(path, context_spec=ts_context)
    
    # Custom collate required if sequences are variable length
    def collate_ragged(batch):
        # Example: Padding to longest in batch
        return torch.nn.utils.rnn.pad_sequence(batch, batch_first=True)

    return DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        collate_fn=collate_ragged,
        pin_memory=True, # Essential for transfer to H100
        persistent_workers=True # Keep TensorStore handles alive
    )

In [ ]:
dl = get_dataloader(path)

In [ ]:
next(iter(dl))

/home/chcharlton/miniforge3/envs/smrt-foundation/lib/python3.12/site-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


KeyboardInterrupt: 

In [ ]:
# import torch
# import tensorstore as ts
# import numpy as np
# from torch.utils.data import Dataset

# class GenomicZarrDataset(Dataset):
#     def __init__(self, zarr_path, feature_shape=(8,)):
#         self.zarr_path = zarr_path
#         self.feature_shape = feature_shape
        
#         # We load the indptr in __init__ because it is safe (pure numpy) 
#         # and efficient (Copy-On-Write means workers share this RAM for free).
#         spec_indptr = {
#             'driver': 'zarr3',
#             'kvstore': {'driver': 'file', 'path': f"{zarr_path}/indptr"}
#         }
#         self.indptr = ts.open(spec_indptr).result().read().result()
#         self.num_reads = len(self.indptr) - 1
        
#         # DO NOT open self.data_store here. 
#         # We want each worker to open its own fresh connection.
#         self.data_store = None

#     def _init_worker_store(self):
#         """Helper to open store only when needed"""
#         if self.data_store is None:
#             spec_data = {
#                 'driver': 'zarr3',
#                 'kvstore': {
#                     'driver': 'file',
#                     'path': os.path.join(os.path.abspath(zarr_path), 'indptr')
#                 }
#             }
#             # Open efficiently without reading yet
#             self.data_store = ts.open(spec_data).result()

#     def __len__(self):
#         return self.num_reads

#     def __getitem__(self, idx):
#         # 1. Ensure this specific worker has a connection
#         self._init_worker_store()
        
#         # 2. Get indices from shared memory
#         start = self.indptr[idx]
#         end = self.indptr[idx+1]
        
#         # 3. Read (This blocks ONLY this worker)
#         data_np = self.data_store[:, start:end].read().result()
        
#         # 4. Return
#         return torch.from_numpy(data_np).T

In [ ]:
import os

class GenomicZarrDataset(Dataset):
    def __init__(self, zarr_path, feature_shape=(8,)):
        # Ensure path is absolute to be safe
        self.zarr_root = os.path.abspath(zarr_path) 
        self.feature_shape = feature_shape
        
        # Open indptr by pointing to root + internal path
        spec_indptr = {
            'driver': 'zarr3',
            'kvstore': {
                'driver': 'file',
                'path': self.zarr_root  # Points to ob007.zarr
            },
            'path': 'indptr'  # Points to internal array 'indptr'
        }
        self.indptr = ts.open(spec_indptr).result().read().result()
        self.num_reads = len(self.indptr) - 1
        self.data_store = None

    def _init_worker_store(self):
        if self.data_store is None:
            spec_data = {
                'driver': 'zarr3',
                'kvstore': {
                    'driver': 'file',
                    'path': self.zarr_root # Points to ob007.zarr
                },
                'path': 'data'  # Points to internal array 'data'
            }
            self.data_store = ts.open(spec_data).result()
            
    def __len__(self):
        return self.num_reads

    def __getitem__(self, idx):
        self._init_worker_store()
        
        # 2. Get indices from shared memory
        start = self.indptr[idx]
        end = self.indptr[idx+1]
        
        # 3. Read (This blocks ONLY this worker)
        data_np = self.data_store[:, start:end].read().result()
        
        # 4. Return
        return torch.from_numpy(data_np).T

In [6]:
ds = GenomicZarrDataset('../data/01_processed/ssl_sets/ob007.zarr')

In [7]:
ds[90000].shape

torch.Size([8000, 8])

In [8]:
def collate_ragged(batch):
    """
    Pads sequences to the longest in the batch.
    """
    # batch is a list of Tensors [(L1, F), (L2, F), ...]
    
    # Get lengths for masking later if needed
    lengths = torch.tensor([x.size(0) for x in batch], dtype=torch.long)
    
    # Pad sequences (batch_first=True -> [B, MaxLen, F])
    # padding_value=0 is standard, adjust if your padding token is different
    padded_batch = torch.nn.utils.rnn.pad_sequence(batch, batch_first=True, padding_value=4)
    
    return padded_batch, lengths
dl = DataLoader(
        ds, 
        batch_size=32, 
        shuffle=False, # Sequential access as planned
        num_workers=4,
        collate_fn=collate_ragged,
        pin_memory=False, # Critical for H100 throughput
        prefetch_factor=2
    )

In [9]:
from tqdm import tqdm

for batch in tqdm(dl):
    x = batch

  0%|          | 0/6141 [00:45<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
import torch
import tensorstore as ts
import numpy as np
import os
from torch.utils.data import IterableDataset, DataLoader
from itertools import islice

class StreamGenomicDataset(IterableDataset):
    def __init__(self, zarr_path, feature_shape=(8,), buffer_size_reads=10000):
        self.zarr_root = os.path.abspath(zarr_path)
        self.feature_shape = feature_shape
        self.buffer_size_reads = buffer_size_reads # How many reads to fetch per I/O call

        # Load indptr completely into RAM (Shared across workers via COW)
        spec_indptr = {
            'driver': 'zarr3',
            'kvstore': {'driver': 'file', 'path': self.zarr_root},
            'path': 'indptr'
        }
        self.indptr = ts.open(spec_indptr).result().read().result()
        self.total_reads = len(self.indptr) - 1
        
    def _get_worker_info(self):
        """Calculates which range of reads this specific worker is responsible for."""
        worker_info = torch.utils.data.get_worker_info()
        if worker_info is None:
            # Single process loading
            return 0, self.total_reads
        else:
            # Split total reads equally among workers
            per_worker = int(np.ceil(self.total_reads / worker_info.num_workers))
            worker_id = worker_info.id
            start = worker_id * per_worker
            end = min(start + per_worker, self.total_reads)
            return start, end

    def __iter__(self):
        # 1. Setup Worker Store
        spec_data = {
            'driver': 'zarr3',
            'kvstore': {'driver': 'file', 'path': self.zarr_root},
            'path': 'data'
        }
        data_store = ts.open(spec_data).result()
        
        # 2. Determine Range
        start_read_idx, end_read_idx = self._get_worker_info()
        
        # 3. Iterate in huge blocks (Vectorized I/O)
        current_idx = start_read_idx
        
        while current_idx < end_read_idx:
            # Define the batch of READS we want to load
            batch_end = min(current_idx + self.buffer_size_reads, end_read_idx)
            
            # Lookup the BYTE positions
            # We grab the relevant slice of indptr to avoid repeated lookups
            # shape: (buffer_size + 1,)
            ptr_batch = self.indptr[current_idx : batch_end + 1]
            
            byte_start = ptr_batch[0]
            byte_end = ptr_batch[-1]
            
            # --- THE OPTIMIZATION ---
            # Single massive read/decompression from TensorStore
            # This aligns well with your 20MB chunks
            buffer_data = data_store[:, byte_start:byte_end].read().result()
            
            # Normalize pointers to be relative to the buffer (starts at 0)
            local_ptr = ptr_batch - byte_start
            
            # Yield individual samples from the RAM buffer
            for i in range(len(local_ptr) - 1):
                p_start = local_ptr[i]
                p_end = local_ptr[i+1]
                
                # Zero-copy view into the buffer
                sample = buffer_data[:, p_start:p_end]
                
                # Transpose to (Seq_Len, Features)
                yield torch.from_numpy(sample).T
            
            current_idx = batch_end

In [ ]:
# Create dataset
ds = StreamGenomicDataset(
    '../data/01_processed/ssl_sets/ob007.zarr', 
    buffer_size_reads=5000 # Adjust this to match roughly 1 or 2 chunks (20-40MB)
)

# Create Loader
dl = DataLoader(
    ds, 
    batch_size=32,    # PyTorch batches up the yielded items
    num_workers=4,    # Each worker streams a different part of the file
    collate_fn=collate_ragged,
    pin_memory=False,  # Now safe to use
)

# Benchmark
for batch in tqdm(dl):
    pass

0it [00:00, ?it/s]


KeyboardInterrupt: 

In [10]:
import torch
import tensorstore as ts
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader

class VectorizedGenomicDataset(Dataset):
    def __init__(self, zarr_path, block_size=1024):
        """
        block_size: Number of reads to fetch in a single I/O call.
                    Set this so that block_size * avg_read_len approx equals your Zarr Chunk size.
                    E.g., 1000 reads * 15kb = 15MB.
        """
        self.zarr_root = os.path.abspath(zarr_path)
        self.block_size = block_size
        
        # Load indptr
        spec_indptr = {
            'driver': 'zarr3',
            'kvstore': {'driver': 'file', 'path': self.zarr_root},
            'path': 'indptr'
        }
        self.indptr = ts.open(spec_indptr).result().read().result()
        self.total_reads = len(self.indptr) - 1
        
        # Calculate how many "blocks" of data exist
        self.num_blocks = int(np.ceil(self.total_reads / self.block_size))
        self.data_store = None

    def _init_worker_store(self):
        if self.data_store is None:
            spec_data = {
                'driver': 'zarr3',
                'kvstore': {'driver': 'file', 'path': self.zarr_root},
                'path': 'data'
            }
            self.data_store = ts.open(spec_data).result()

    def __len__(self):
        return self.num_blocks

    def __getitem__(self, idx):
        self._init_worker_store()
        
        # 1. Calculate the range of READS for this block
        read_start_idx = idx * self.block_size
        read_end_idx = min((idx + 1) * self.block_size, self.total_reads)
        
        # 2. Get the range of BYTES from indptr
        # We grab the slice of indptr corresponding to these reads
        # ptrs shape: (n_reads_in_block + 1)
        ptrs = self.indptr[read_start_idx : read_end_idx + 1]
        
        byte_start = ptrs[0]
        byte_end = ptrs[-1]
        
        # 3. The "Big Read"
        # One single C++ call to fetch ~15MB of data
        # This matches your original Zarr chunking strategy perfectly
        raw_block = self.data_store[:, byte_start:byte_end].read().result()
        
        # 4. Slice in RAM (CPU is fast here)
        # Normalize pointers to be relative to the block
        local_ptrs = ptrs - byte_start
        
        reads = []
        for i in range(len(local_ptrs) - 1):
            p0, p1 = local_ptrs[i], local_ptrs[i+1]
            # Zero-copy slice
            sample = raw_block[:, p0:p1]
            # Transpose to (Seq, Feat) for PyTorch
            reads.append(torch.from_numpy(sample).T)
            
        return reads

In [11]:
def collate_flattened(batch_of_blocks):
    """
    batch_of_blocks: List[List[Tensor]]
    
    Structure:
    [
      [Read_0, Read_1, ... Read_1023], # From Worker 1
      [Read_1024, ..., Read_2047],     # From Worker 2
      ...
    ]
    """
    # 1. Flatten the list of lists into one giant list of reads
    flat_reads = [read for block in batch_of_blocks for read in block]
    
    # 2. Pad and Stack
    lengths = torch.tensor([x.size(0) for x in flat_reads], dtype=torch.long)
    padded_batch = torch.nn.utils.rnn.pad_sequence(flat_reads, batch_first=True, padding_value=0)
    
    return padded_batch, lengths

In [12]:
# 1. Use the original Zarr file (Large Chunks)
ds = VectorizedGenomicDataset(
    '../data/01_processed/ssl_sets/ob007.zarr', 
    block_size=2000 # Adjust this so block_size * avg_len ~= 20MB
)

# 2. DataLoader Configuration
# effective_batch_size = batch_size * block_size
# If batch_size=4 and block_size=2000, you get 8000 reads per step.
dl = DataLoader(
    ds, 
    batch_size=4,       # Small number here, because each item is HUGE
    num_workers=4,      # 4 workers reading in parallel
    collate_fn=collate_flattened,
    pin_memory=True,
    prefetch_factor=2
)

# Benchmark
for x, lengths in tqdm(dl):
    # x shape: [8000, Max_Len, Features]
    pass

  0%|          | 0/25 [00:00<?, ?it/s]/home/chcharlton/miniforge3/envs/smrt-foundation/lib/python3.12/site-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
  0%|          | 0/25 [00:49<?, ?it/s]


KeyboardInterrupt: 